<a href="https://www.kaggle.com/code/theyazilimci/alzheimer-prediction-92-acc?scriptVersionId=93719730" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Alzheimer Prediction using Convolutional Neural Network 



<div class="alert alert-block alert-info" 
     style="font-size:16px; font-family:Helvetica;
            border-radius: 10px;">
<font size="3">
    In this notebook we'll try to predict wheter or not a person have the alzheimer depending on the dataset.
We can see 4 folders that contains differents images with a total size of 44.1MB.<br> To classify those image we'll using tensorflow framework.<br> The dataset aren't split between training and testing set so we will do it because model can perform well with the training set however it can make bad prediction on something that it he never see before
 </font>

</div>
    
![image](https://c4.wallpaperflare.com/wallpaper/353/981/279/brain-splitting-selective-coloring-musical-notes-wallpaper-preview.jpg)

<font size=5> Data Importation 📖 </font>

In [ ]:
import tensorflow as tf 
import matplotlib.pyplot as plt 
import numpy as np 
import os 
import pathlib 
import random

In [ ]:
path = '../input/alzheimer-mri-dataset/Dataset/'
data_dir = pathlib.Path(path)

<font size=5> Getting class names ♑︎ </font>

In [ ]:
class_names = np.array([sorted(item.name for item in data_dir.glob("*"))])
class_names

<font size=5> Total number of images 💯 </font>

In [ ]:
imageCount = len(list(data_dir.glob("*/*.jpg") ))
imageCount

<font size=5> Example Image 📈 </font>

In [ ]:
def plot(path,class_name):
    print(path)
    plt.figure(figsize=(8,8))
    
    img = plt.imread(path)
    
    plt.xticks([])
    plt.yticks([])
    plt.title("Class Name: "+class_name)
    plt.imshow(img)

In [ ]:
Mild_Demented = random.choice(list(data_dir.glob("Mild_Demented/*.jpg")))

plot(str(Mild_Demented),"Mild_Demented")

In [ ]:
Moderate_Demented = random.choice(list(data_dir.glob("Moderate_Demented/*.jpg")))

plot(str(Moderate_Demented),"Moderate_Demented")

In [ ]:
Non_Demented = random.choice(list(data_dir.glob("Non_Demented/*.jpg")))

plot(str(Non_Demented),"Non_Demented")

In [ ]:
Very_Mild_Demented = random.choice(list(data_dir.glob("Very_Mild_Demented/*.jpg")))

plot(str(Very_Mild_Demented),"Very_Mild_Demented")

<div class="alert alert-block alert-info" 
     style="font-size:16px; font-family:Helvetica;
            border-radius: 10px;">
<font size="4">
    We can see the difference between those classes even if some images have the same shame like a snack the model will perform well to predict the moderate demented because we can see very easily the difference between the moderate and the other one 
    </font>

</div>

<font size=5> Split the Dataset 🎬 </font>

In [ ]:
batch_size = 32
img_height = 224
img_width = 224

In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.utils import image_dataset_from_directory

train_data = image_dataset_from_directory(
                  data_dir,
                  validation_split=0.2,
                  subset="training",
                  seed=123,
                  image_size=(img_height, img_width),
                  batch_size=batch_size)


val_data = image_dataset_from_directory(data_dir,
                                        validation_split=0.2,
                                        subset="validation",
                                        seed=123,
                                        image_size=(img_height,img_width),
                                        batch_size=batch_size)

<font size=5> Build the model 🏗 </font>
    
<div class="alert alert-block alert-info" 
     style="font-size:16px; font-family:Helvetica;
            border-radius: 10px;">
<font size="3">
    <b>Explanation</b>: We rescale the images to fit between [0,1] then for each convolutional layer we can see that the size increase 16-32-64 the parameter 3 is the kernel size often it's 3 but we can see 5 too we specify the padding as same this mean when the convolution occurs we put all around the image 0 pixels because when you are making convolution the kernel have to be in the center of all the pixels that it see, if we do not add padding it cannot convolute the edges so we will loose some information,The max pooling have a size of 2 by 2 matrix it parse the images and get the maximum value between 4 pixel and so on....  at the end we add a dropout layer to avoid overfit this will off some neurons randomly then we add a flatten layer to put all this in one dimensional array, the activation is relu except for the last layer that have to return us a probability of belonging to each class so the last layer have to have as neurons the number of classes 
    </font>

</div>

In [ ]:
from tensorflow.keras import layers

model = tf.keras.Sequential([
    
   layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
    
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
    
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
    
  layers.Dropout(0.5),
  layers.Flatten(),
    
  layers.Dense(128, activation='relu'),
  layers.Dense(4,activation="softmax")
])

<font size=5> Compile the model 💿</font>

Difference between Sparse and Categorical: https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&ved=2ahUKEwiw66__1af3AhVHTBoKHQCYB7MQFnoECAQQAQ&url=https%3A%2F%2Fdatascience.stackexchange.com%2Fquestions%2F41921%2Fsparse-categorical-crossentropy-vs-categorical-crossentropy-keras-accuracy&usg=AOvVaw0fPR_3m2cNqRyLbT4kwDw0

In [ ]:
model.compile(optimizer="Adam",
            loss=tf.keras.losses.SparseCategoricalCrossentropy(),
            metrics=["accuracy"])

<font size=5> Fit the model 🏋🏼 
    <b>92% Accuracy </b></font>

In [ ]:
epochs = 5 
history = model.fit(train_data,
                    epochs=epochs,
                    validation_data=val_data, 
                    batch_size=batch_size)

<font size=5> Plot the result 📈  </font>

In [ ]:
acc = history.history['accuracy']
val_acc =  history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
plt.plot(epochs_range,acc,label='Accuracy')
plt.plot(epochs_range,val_acc,label="Validation Accuracy")
plt.legend()

plt.subplot(1,2,2)
plt.plot(epochs_range,loss,label='Loss')
plt.plot(epochs_range,val_loss,label="Validation Loss")
plt.legend()
plt.show()

<font size=5> Predictions 🧙</font>

In [ ]:
plt.figure(figsize=(20, 20))
class_names = val_data.class_names
result = ' | False'
for images, labels in val_data.take(1):
    for i in range(25):
        
        ax = plt.subplot(5, 5, i + 1)
        
        img = images[i].numpy().astype("uint8")
        img = tf.expand_dims(img, axis=0)
        
        predictions = model.predict(img)
        predicted_class = np.argmax(predictions)
        if class_names[predicted_class] == class_names[labels[i]]:
            result = ' | TRUE'
            
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[predicted_class]+result  )
        plt.axis("off")

![image](https://c4.wallpaperflare.com/wallpaper/353/981/279/brain-splitting-selective-coloring-musical-notes-wallpaper-preview.jpg)